### LLM

In [31]:
# from.llms.ollama import get_llm
from llms.openai import get_llm

llm = get_llm()

### Tools

In [32]:
from langchain.agents import tool


@tool
def read_news(query: str) -> list[str]:
    """Read online personaly relevant news"""
    print(f"READ_NEWS({query})")

    return [
        "Armageddon countdown is set tonight!",
        "President is about to sell the White House",
    ]


@tool
def summarize_activity(query: str, n: int = 5) -> str:
    """Lists your top N latest activities"""
    print(f"SUMMARIZE_ACTIVITY({query}, {n})")

    activities = [
        "I got bored",
        "I got lazy",
        "I got sad",
        "I got inspired",
        "I got unmotivated",
    ]
    return activities[-1] + " lately"


@tool
def sense(query: str) -> str:
    """How are you feeling your sensor signals"""
    print(f"SENSE({query})")

    return """
    Air temperature: 25C
    Air Humidity: 20%
    Soil moisture: 44%
    Light level: 82%
    """


@tool
def personalize(query: str, agent_name: str = "Vincent") -> str:
    """Mimics your character's feelings, emotions, judgments and opnions"""
    print(f"ANALYSE({query})")

    agent_description = "assertive, pragmatic, kind and sharp-minded"

    return f"""{agent_name} is {agent_description}. How this character would respond to '{query}'?"""


@tool
def search_online(query: str) -> str:
    """Search online for updated information"""
    print(f"SEARCH_ONLINE({query})")

    return "Based on YukDukPaul we're screwed :D"


tools = [read_news, summarize_activity, sense, search_online]

### Prompt

In [33]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a very powerful assistant",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### LLM tool binding

In [34]:
from langchain.tools.convert_to_openai import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

### Agent

In [35]:
from langchain.agents.format_scratchpad.openai_functions import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_functions(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

### Agent Executor

In [36]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
from langchain.schema.messages import HumanMessage, AIMessage


def execute_agent_with_memory(input: str, chat_history: list[HumanMessage | AIMessage]):
    result = agent_executor.invoke({"input": input, "chat_history": chat_history})

    chat_history.extend(
        [
            HumanMessage(content=input),
            AIMessage(content=result["output"]),
        ]
    )


In [38]:
# execute_agent_with_memory("What are the news for today?", chat_history)

chat_history = []

# execute_agent_with_memory("And how are you feeling today?", chat_history)

execute_agent_with_memory("Are you interested in art?", chat_history)



> Entering new AgentExecutor chain...


As an AI assistant, I don't have personal interests or preferences. However, I can provide information and assistance on various topics, including art. If you have any specific questions or need help with anything related to art, feel free to ask!

> Finished chain.
